**In this project, I will process labelled news headlines, perform topic modelling, and then build predictive models to classify news headlinesinto different domains: Business, SciTech, Sports, World.**

In [4]:
import pandas as pd
import numpy as np
import spacy
import re
import json
import itertools as it

'''Visualization'''
import matplotlib.pyplot as plt
import seaborn as sns

''' Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

'''Estimators'''
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

'''Modelling'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_curve, auc, classification_report

## Import data notes

In [5]:
# Take a look at what the first headline looks like
with open("news_article_data.json") as file:
    first_record = file.readline()

print(first_record)  # two fields are of interest: "content" and "annotation".

{"content": "Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.","annotation":{"notes":"","label":["Business"]},"extras":null,"metadata":{"first_done_at":1521027375000,"last_updated_at":1521027375000,"sec_taken":0,"last_updated_by":"nlYZXxNBQefF2u9VX52CdONFp0C3","status":"done","evaluation":"NONE"}}



**Write all headlines that are labelled "Business" in a text file.**

In [6]:
import os

scratch_directory = "scratch"

# create a scratch directory if one doesn't already exist
try:
    os.mkdir(scratch_directory)
except FileExistsError:
    pass

# path for our text file that contains all the business headlines
business_txt_file_path = os.path.join(scratch_directory,"business_headlines_all.txt")

In [7]:
execute = True   # can turn this flag to True if I want to run this cell

if execute:
    business_count = 0 # count the number of business headlines
    
    # create and open a new file in write mode
    with open(business_txt_file_path,'w') as business_txt_file:
        
        
        # open the news headline data file in read mode
        with open ("news_article_data.json",'r') as headlines_file:
            
            # loop through all rows in the headline file and convert each row into a dict
            for line in headlines_file:
                headline = json.loads(line)
                
                # if the headline is not labelled "Business", go back to the start of the loop for the next headline
                if headline['annotation']['label'][0] !="Business":
                    continue
                
                # write the news article that is labelled "Business" in the new file
                business_txt_file.write(headline['content']+'\n' + '\n')
                business_count +=1 # increment count
    
    print("The number of news articles written into business_headlines_all.txt is {}".format(business_count))


The number of news articles written into business_headlines_all.txt is 1900


## Import data

In [8]:
# Read in each headline and convert it to a Python dict, 
# get the text of each headline, and write it into a new text file that  
# contains  one headline per line in the file.

scratch_directory = os.path.join('scratch')

# create a scratch directory if one doesn't already exist
try:
    os.mkdir(scratch_directory)
except FileExistsError:
    pass

headline_txt_filepath = os.path.join(scratch_directory, 'headline_text_all.txt') # name of the text file that contains all headline text

In [9]:
headline_count = 0 #  init headline counter
label = [] # a list for the headline labels

# create and open a new file in write mode
with open(headline_txt_filepath, 'w') as headline_txt_file:
    
    # open "news_article_data.json" in read mode (the original data file)
    with open ("news_article_data.json", 'r') as headline_json_file:
        
        # convert each json record into dict and write the headline text into the new file
        for line in headline_json_file:
            headline = json.loads(line)
            headline_txt_file.write(headline['content'] + '\n')
            headline_count +=1
            
            #Get label
            label.append(headline['annotation']['label'][0])  
            

In [10]:
len(label), headline_count

(7600, 7600)

************************************************************************************************************

## SpaCy Notes

In [11]:
nlp = spacy.load('en_core_web_sm')

**Grab a sample headline to "play" with and try out the functionalities of spaCy.**

In [12]:
headline_num = 789

with open("news_article_data.json") as file:
    # convert the json record to a Python dict. "data" is a list of dicts
    data=[json.loads(line) for line in file]
    
sample_headline = data[headline_num]['content']
print(sample_headline)

Bea Arthur sparked a security scare at Logan Airport in Boston this week when she tried to board a Cape Air flight with a pocketknife in her handbag.    The "Golden Girls" star, now 81, was flagged by a Transportation Security Administration agent, who discovered the knife - a strict no-no following 9/11.    "She started yelling that it wasn't hers and said 'The terrorists put it there,' " a fellow passenger said. "She kept yelling about the 'terrorists, the terrorists, the terrorists.' "    After the blade was confiscated, Arthur took a keyring from her bag and told the agent it belonged to the "terrorists," before throwing it at them.  - via philly.com


In [13]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [14]:
parsed_headline = nlp(sample_headline) # apply spaCy nlp models to the sample headline (ex: tagger, parser, ner )

In [15]:
print(parsed_headline)

Bea Arthur sparked a security scare at Logan Airport in Boston this week when she tried to board a Cape Air flight with a pocketknife in her handbag.    The "Golden Girls" star, now 81, was flagged by a Transportation Security Administration agent, who discovered the knife - a strict no-no following 9/11.    "She started yelling that it wasn't hers and said 'The terrorists put it there,' " a fellow passenger said. "She kept yelling about the 'terrorists, the terrorists, the terrorists.' "    After the blade was confiscated, Arthur took a keyring from her bag and told the agent it belonged to the "terrorists," before throwing it at them.  - via philly.com


**Looks the same so far!  But spaCy already performed all the sentence detection, tokenization, normalization etc...**

**Sentence Detection**

In [16]:
parsed_headline.sents # sentencize the sample headline, I get a generator

In [17]:
for i, text in enumerate(parsed_headline.sents):
    print("Sentence number:{}".format(i))
    print(text)
    print('')

Sentence number:0
Bea Arthur sparked a security scare at Logan Airport in Boston this week when she tried to board a Cape Air flight with a pocketknife in her handbag.    

Sentence number:1
The "Golden Girls" star, now 81, was flagged by a Transportation Security Administration agent, who discovered the knife - a strict no-no following 9/11.    

Sentence number:2
"She started yelling that it wasn't hers and said 'The terrorists put it there,' " a fellow passenger said.

Sentence number:3
"She kept yelling about the 'terrorists, the terrorists, the terrorists.' "    

Sentence number:4
After the blade was confiscated, Arthur took a keyring from her bag and told the agent it belonged to the "terrorists," before throwing it at them.  

Sentence number:5
- via philly.com



**Tokenization, lemmatization, and token shape analysis**

In [18]:
token_text = [token.text for token in parsed_headline] 
token_lemma = [token.lemma_ for token in parsed_headline] 
token_shape = [token.shape_ for token in parsed_headline]

In [19]:
temp= pd.DataFrame(zip(token_text, token_lemma, token_shape),
            columns= ['token text', 'token lemma', 'token shape'])

temp.head(10)

,token text,token lemma,token shape
0,Bea,Bea,Xxx
1,Arthur,Arthur,Xxxxx
2,sparked,spark,xxxx
3,a,a,x
4,security,security,xxxx
5,scare,scare,xxxx
6,at,at,xx
7,Logan,Logan,Xxxxx
8,Airport,Airport,Xxxxx
9,in,in,xx


**spaCY also performs part of speech tagging (detects the verb, noun, proper noun etc... in a sentence)**
- Noun (N)- Daniel, London, table, dog, teacher, pen, city, happiness, hope
- Verb (V)- go, speak, run, eat, play, live, walk, have, like, are, is
- Adjective(ADJ)- big, happy, green, young, fun, crazy, three
- Adverb(ADV)- slowly, quietly, very, always, never, too, well, tomorrow
- Preposition (P)- at, on, in, from, with, near, between, about, under
- Conjunction (CON)- and, or, but, because, so, yet, unless, since, if
- Pronoun(PRO)- I, you, we, they, he, she, it, me, us, them, him, her, this
- Interjection (INT)- Ouch! Wow! Great! Help! Oh! Hey! Hi!

In [20]:
token_pos = [token.pos_ for token in parsed_headline]
temp = pd.DataFrame(
    zip(token_text, token_pos),
    columns=['token_text', 'part_of_speech']
    )

temp.sample(10)

,token_text,part_of_speech
54,a,DET
14,she,PRON
103,',PUNCT
5,scare,NOUN
26,her,DET
134,at,ADP
44,Transportation,PROPN
55,strict,ADJ
90,kept,VERB
22,with,ADP


**Name Entity Recognition (NER)**
- A name entity is a "real world object" that is assigned a name.  For example:
    - A person
    - A country
    - A product
    - A book title
        
- If my text data is a chapter of a fiction, and my task is to find out the character that appeared the most in that chapter, then I need NER.
- First, I need to identify the names in a sentence.
- Then I need to get a list of all the unique names that appear in all the sentences in the chapter.
- Finally, I count the number of times that each name appear.

In [21]:
parsed_headline

Bea Arthur sparked a security scare at Logan Airport in Boston this week when she tried to board a Cape Air flight with a pocketknife in her handbag.    The "Golden Girls" star, now 81, was flagged by a Transportation Security Administration agent, who discovered the knife - a strict no-no following 9/11.    "She started yelling that it wasn't hers and said 'The terrorists put it there,' " a fellow passenger said. "She kept yelling about the 'terrorists, the terrorists, the terrorists.' "    After the blade was confiscated, Arthur took a keyring from her bag and told the agent it belonged to the "terrorists," before throwing it at them.  - via philly.com

In [22]:
parsed_headline.ents

(Bea Arthur,
 Logan Airport,
 Boston,
 this week,
 Cape Air,
 The "Golden Girls" star,
 81,
 Transportation Security Administration,
 9/11,
 Arthur)

In [23]:
print(type(parsed_headline.ents))

<class 'tuple'>


In [24]:
for i, entity in enumerate(parsed_headline.ents):
    print("Entity {}: {} - {}".format(i+1, entity, entity.label_))

Entity 1: Bea Arthur - PERSON
Entity 2: Logan Airport - FAC
Entity 3: Boston - GPE
Entity 4: this week - DATE
Entity 5: Cape Air - PRODUCT
Entity 6: The "Golden Girls" star - FAC
Entity 7: 81 - DATE
Entity 8: Transportation Security Administration - ORG
Entity 9: 9/11 - CARDINAL
Entity 10: Arthur - PERSON


**Token-level attributes**
- Whether or not a token is a:
    - stopword?
    - punctuation?
    - whitespace?
    - a number?
    - is part of spaCy's default vocabulary?

In [25]:
token_attributes = [(
        token.text,  # text
        token.prob, # log prob (frequency of word)
        token.is_stop, # stop word?
        token.is_punct, # a puncuation?
        token.is_space, # a whitespace?
        token.like_num, # a number?
        token.is_oov # not a spaCy vocab?
        ) for token in parsed_headline
    ]

In [26]:
temp = pd.DataFrame(token_attributes,
       columns=['text', 'log_probability',
        'stop?', 'punctuation?',
        'whitespace?', 'number?',
        'out of vocab.?']
    )

In [27]:
temp.loc[:, 'stop?':'out of vocab.?'] = (
    temp.loc[:, 'stop?':'out of vocab.?']
    .applymap(lambda x: 'Yes' if x else '')
    )
                                               
temp.tail(10)

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
131,before,-20.0,Yes,,,,Yes
132,throwing,-20.0,,,,,Yes
133,it,-20.0,Yes,,,,Yes
134,at,-20.0,Yes,,,,Yes
135,them,-20.0,Yes,,,,Yes
136,.,-20.0,,Yes,,,Yes
137,,-20.0,,,Yes,,Yes
138,-,-20.0,,Yes,,,Yes
139,via,-20.0,Yes,,,,Yes
140,philly.com,-20.0,,,,,Yes


**NOTE:** 

- The intended interpretation of oov is, "tokens that don't have a meaningful probability value are words that are not in the vocab."

- oov is not that useful in the _sm models, and in general might not work well in data packs if there wasn't a big frequency count used to build the vocab.

************************************************************************************************************

## Processing Text
- Use spaCy to remove punctuation and lemmatize the text.
- Train and apply first-order phrase model to join word pairs.
- Train and apply second-order phrase model to join longer phrases.
- Remove stopwords.
- Create a bag-of-words representation.

In [28]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence

In [29]:
nlp = spacy.load('en_core_web_sm')

In [30]:
# This function helps eliminate tokens that are punctutation or whitespace.
def punct_space(token):

    '''
    NOTE: 

    Unlike verbs and common nouns, there is no clear base form of a personal pronoun. 
    It is not clear if the lemma of "me" sould be "I", or whether a person is normalized as "it" or "she" or "he"? 
    spaCy's solution is to introduce a novel symbol, -PRON-, which is used as the lemma for all personal pronouns.
    '''

    # I will return the actual text if I encounter a pronoun instead of using spaCy's -PRON-  Otherwise, I wil have -PRON- all over the headline
    if token.lemma_ == '-PRON-': # if the lemma is -PRON- I just return the lower case of the personal pronoun.
        return token.lower_ 
    else:
        return token.is_punct or token.is_space # otherwise, I return TRUE if token is a punctuation or whitespace, and FALSE if otherwise
    

# This function returns TRUE for alphanumeric characters 
def get_alphanumeric(token):
    return token.is_alpha or token.is_digit

# A generator function that reads headlines from "headline_text_all.txt" (the text file that contains all headline text)
def get_line_headline(filename):
    with open(filename, 'r') as f:
        for headline in f:
            yield headline
            
# This function lemmatizes/normalizes a token (in lowercase)            
def lemm(token):
    return token.lemma_.lower()

In [31]:
lemm_hl_filepath = os.path.join(scratch_directory, 'lemm_headline_txt_all.txt') # text file that contains cleaned and lemmatized text
lemm_hl_sent_filepath = os.path.join(scratch_directory, 'lemm_headline_sentence_all.txt') # contains cleaned, lemmatized, and sentencized text 

In [32]:
%%time
execute = False

if execute:
    
    with open(lemm_hl_filepath, 'w') as lemm_hl_file:   # open lemm_headline_txt_all.txt in write mode
         with open(lemm_hl_sent_filepath, 'w') as lemm_sentence_file: # open lemm_headline_sentence_all.txt in write mode
                
                # stream in the headlines from "headline_txt_all.txt" 500 at a time.
                # nlp.pipe applies nlp to each headline
                pipe = nlp.pipe( 
                    get_line_headline(headline_txt_filepath), # read line for file that contains all headline text (one per line)
                    batch_size = 500, 
                    n_threads=8)
        
                for headline in pipe:
                    # lemmatize each headline (convert from a list of tokens to a string)
                    lemm_headline = ' '.join([lemm(token) for token in headline if (not punct_space(token) and 
                                                                            get_alphanumeric(token))])
            
                    # Save the lemmatized text in "lemm_headline_txt_all.txt"
                    lemm_hl_file.write(lemm_headline + '\n')
            
            
                    # Sentencize each headline, and then lemmatize each sentence
                    for sentence in headline.sents: # sentencize each headline
                        #lemm each sentence
                        lemm_sentence = ' '.join([lemm(token) for token in sentence if (not punct_space(token) and
                                                                               get_alphanumeric(token))])
                
                        #Save each of the lemmatized sentence as a new line in "lemm_headline_sentence_all.txt"
                        lemm_sentence_file.write(lemm_sentence + '\n')

Wall time: 0 ns


- In "lemm_headline_sentence_all.txt", I have one sentence of each headline per line.
- I use gensim LineSentence class to get an iterator of sentences. The LineSentence class allows me to easily work with other gensim components by streaming the docs (sentences) from disk (and not hold entire dataset/corpora in RAM at once).  
- Thus, using LineSentence from gensim, I can to up-scale my modelling pipeline to very large corpora.

In [33]:
sentences_unigrams = LineSentence(lemm_hl_sent_filepath) # a LineStence object that holds all sentences in my dataset

In [34]:
type(sentences_unigrams)

gensim.models.word2vec.LineSentence

**Let's Take a look at a few sample sentences in the transformed corpora.**

In [35]:
for sent_unigram in it.islice(sentences_unigrams, 10, 15):  # look at sentences with index 10 to 20   (it.islice(iterator, start, end, step))
    print(sent_unigram)

['emi', 'have', 'apparently', 'negotiate', 'end', 'well', 'so', 'that', 'child', 'in', 'school', 'will', 'now', 'be', 'indoctrinate', 'about', 'the', 'illegality', 'of', 'download', 'music']
['the', 'ignorance', 'and', 'audacity', 'of', 'this', 'get', 'to', 'a', 'little', 'so', 'write', 'an', 'open', 'letter', 'to', 'the', 'dfes', 'about']
['unfortunately', 'pedantic', 'as', 'suppose', 'have', 'to', 'be', 'when', 'write', 'to', 'goverment', 'representative']
['but', 'hope', 'find', 'useful', 'and', 'perhaps', 'feel', 'inspired', 'to', 'do', 'something', 'similar', 'if', 'or', 'when', 'the', 'same', 'thing', 'have', 'happen', 'in', 'area']
['jaschan', 'self', 'confess', 'author', 'of', 'the', 'netsky', 'and', 'sasser', 'virus', 'for', '70', 'percent', 'of', 'virus', 'infection', 'in', '2004', 'accord', 'to', 'a', 'six', 'roundup', 'publish', 'wednesday', 'by', 'antivirus', 'company', 'sophos']


In [36]:
for sent_unigram in it.islice(sentences_unigrams, 10, 15): 
    print(' '.join(sent_unigram)) # convert to string
    print('') # add space between sentences

emi have apparently negotiate end well so that child in school will now be indoctrinate about the illegality of download music

the ignorance and audacity of this get to a little so write an open letter to the dfes about

unfortunately pedantic as suppose have to be when write to goverment representative

but hope find useful and perhaps feel inspired to do something similar if or when the same thing have happen in area

jaschan self confess author of the netsky and sasser virus for 70 percent of virus infection in 2004 accord to a six roundup publish wednesday by antivirus company sophos



## Phrase Modelling
- Learn combinations of tokens that together represents meaningful multi-word concepts (ex: "United States", "happy hour")
- These phrase models are developed by examining all the words in The headlines and looking for words that co-occur (i.e., appear one after another) together much more frequently than you would expect them to by random chance. 

- Two important parameters for phrase models:
    - $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times.
    - $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase.

- Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.
- We would get multi-word expressions that represent common concepts, but aren't specifically named entities (ex: happy hour, in addition to named entities such as New York) to become phrases in the model.

**Finding Bigrams (first order phrase model)**
- Train a phrase model using the headline corpora to identify/link words into 2-word phrases (or bigrams).  For example, words that often appear together in the corpora will be linked together into a single token (ex: happy hour==> happy_hour)  

In [37]:
bigram_model_filepath = os.path.join(scratch_directory, 'bigram_phrase_model') 

In [38]:
# Train the bigram model
execute = False
if execute:
    # train the phrase model using the processed sentences (LineSentence object)
    bigram_phrases_model = Phrases(sentences_unigrams)
    # Use the Phraser function to turn the phrase model into a "Phraser" object, which is optimized for speed and memory use
    bigram_phrases_model = Phraser(bigram_phrases_model)
    #Save the model for future use
    bigram_phrases_model.save(bigram_model_filepath)

In [39]:
# Load the trained model from disk
bigram_phrases_model = Phraser.load(bigram_model_filepath)

In [40]:
bigram_phrases_model

**Apply the trained Phrase model to our lemmatized sentences**

In [41]:
# new file that contains transformed sentences (with bigrams)
sentences_bigrams_filepath = os.path.join(scratch_directory, 'sentence_bigram_phrases_all.txt')

In [42]:
execute = False

if execute:
    # write the sentences (with bigrams) in "sentence_bigram_phrases_all.txt"
    with open(sentences_bigrams_filepath, 'w') as bigram_file:
        for sent in sentences_unigrams: # sent is a list of tokens
            sent_with_bigram = ' '.join(bigram_phrases_model[sent])  # apply the phrase model to each lemmatized sentences
            bigram_file.write(sent_with_bigram + '\n') # write each transformed sentences into a text file

**We see two-word phrases, such as "plan_to" linked together in the text as a single token.**

**The first-order transformed data is now organized with one document (sentence) per line in "sentence_bigram_phrases_all.txt".  As before, I use the gensim LineSentence class to get an LineSentence object for the entire set of documents.**

In [43]:
sentences_bigrams = LineSentence(sentences_bigrams_filepath)

In [44]:
type(sentences_bigrams)

gensim.models.word2vec.LineSentence

**Finding Trigrams (second order phrase model)**
- I apply the second-order phrase model on top of the transformed data in "sentence_bigrams_phrases_all.txt"), so that incomplete word combinations like "big bad_wolf" will become fully joined to "big_bad_wolf".

**Train trigram model**

In [45]:
# new filepath for transformed sentences (with trigrams)
trigram_model_filepath = os.path.join(scratch_directory, 'trigram_phrase_model')

In [46]:
execute = False
if execute:
    trigram_phrases_model = Phrases(sentences_bigrams)
    trigram_phrases_model = Phraser(trigram_phrases_model)
    
    # Save model
    trigram_phrases_model.save(trigram_model_filepath)

In [47]:
# Load the trained model from disk
trigram_phrases_model = Phraser.load(trigram_model_filepath)

**Apply trained trigram phrase model to our "first-order transformed sentences" (aka sentences with bigrams). Then we will write the "second-order transformed sentences" (aka sentences with trigrams) into a file.**

In [48]:
sentences_trigrams_filepath = os.path.join(scratch_directory, 'sentence_trigram_phrases_all.txt')

In [49]:
execute = True
if execute:
    with open(sentences_trigrams_filepath, 'w') as trigram_file:
        for sent_bigram in sentences_bigrams: # iterate the LineSentence object that contains the set of docs with bigrams
            sent_trigram = ' '.join(trigram_phrases_model[sent_bigram])
            
            # write the transformed sentence into 'sentence_trigram_phrases_all.txt'
            trigram_file.write(sent_trigram + '\n')

**The second-order transformed data is now organized with one document (sentence) per line in "sentence_trigram_phrases_all.txt". As before, I use gensim LineSentence class to get a LineSentence object for the entire set of documents.**

In [50]:
sentences_trigrams = LineSentence(sentences_trigrams_filepath)

## Process headlines
- Now that I have the trained bigram and trigram phraser models (from each individual sentences in my set of headlines), I will now process the lemmatized headlines data (that is not broken down into individual sentences).
- I will first create the gensim LineSentence object for the lemmatized headlines corpus.
- Second, I will apply the first and second order transformation to the headlines.
- Third, I will remove stopwords. Stopwords are very common words, like a, the, and, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text (adds only noise). Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.
- Finally, I will write the transformed headlines (with trigrams, one for each line) into a file.

In [51]:
headlines_trigrams_filepath = os.path.join(scratch_directory, 'headlines_trigrams_all.txt')

In [52]:
execute = True
if execute:
    # create the LineSentence iterator for the lemmatized headlines
    lemm_headlines = LineSentence(lemm_hl_filepath)
    
    # write the transformed headlines into "headlines_trigrams_all.txt"
    with open(headlines_trigrams_filepath, 'w') as headlines_trigram_file:
        
        for hl_unigram in lemm_headlines: # get each headline from the lemm_headlines LineSentence object
            # apply the trained bigram and trigram phrase models to each headline
            hl_bigram = bigram_phrases_model[hl_unigram]
            hl_trigram = trigram_phrases_model[hl_bigram]
            
            # remove stopwords from each of the transformed headline
            hl_trigram = [t for t in hl_trigram if t not in nlp.Defaults.stop_words]
            
            # convert the list of tokens into a single string for each transformed headline
            hl_trigram = ' '.join(hl_trigram)
            
            # write the transformed headline (with trigram) into "headlines_trigrams_all.txt"
            headlines_trigram_file.write(hl_trigram + '\n')            

**Compared the original headline and transformed headline**
- What I did?
    - Tokenized and lemmatized text.
    - Removed stopwords, punctuations, and non-alphanumeric characters.
    - Include bigrams and trigrams.

In [53]:
headline_num = 200

with open("news_article_data.json") as file:  
    data=[json.loads(line) for line in file]
    
sample_headline = data[headline_num]['content']
print('Original:' + '\n')
print(sample_headline)


print('\n')
print('Transformed:' + '\n')

with open(headlines_trigrams_filepath) as file:
    for transformed_sample in it.islice(file, headline_num, headline_num+1):
        print(transformed_sample)

Original:

Single mothers, poverty were big factors in school performance    HealthDayNews -- American teenagers who live with poor single mothers are more likely to get into trouble at school and have poor marks and are less likely to think they'll go to college, says a Rice University study.    Holly Heard, an assistant professor of sociology, analyzed data from thousands of teens who took part in the National Longitudinal Study of Adolescent Health...


Transformed:

single mother poverty big factor school performance healthdaynew american teenager live poor single mother likely trouble school poor mark likely think college rice university study holly heard assistant professor sociology analyze datum thousand_of teen take_part national longitudinal study adolescent health



## Topic Modelling
- Topic modeling is a method for unsupervised classification of documents, similar to clustering on numeric data, which finds some natural groups of items (topics) even when we’re not sure what we’re looking for.
- It is similar to clustering for numerical data.
- Topic modeling provides methods for automatically organizing, understanding, searching, and summarizing large electronic archives.
    - Discovering the hidden themes in the collection.
    - Classifying the documents into the discovered themes.
    - Using the classification to organize/summarize/search the documents.
    - For example, let’s say a document belongs to the topics food, dogs and health. So if a user queries “dog food”, they might find the above-mentioned document relevant because it covers those topics(among other topics).  Therefore, by annotating the document, based on the topics predicted by the modeling method, we are able to optimize our search process.

## LDA basics:
https://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/
- One of the most popular topic modeling methods. 
-  Each document is made up of various words, and each topic also has various words belonging to it. 
-  The aim of LDA is to find topics a document belongs to, based on the words in it.
- Assumptions:
    - We know beforehand how many topics we want. ‘k’ is pre-decided.
    - Each document is just a collection of words or a “bag of words”.
   
- Two parts in LDA:
    - The words that belong to a document, that we already know.
    - The words that belong to a topic or the probability of words belonging into a topic, that we need to calculate.
- We use an algorithm to find the latter.

- LDA represents documents as a mixture of topics. Similarly, a topic is a mixture of words. If a word has high probability of being in a topic, all the documents having w will be more strongly associated with t as well. 
- Similarly, if w is not very probable to be in t, the documents which contain the w will be having very low probability of being in t, because rest of the words in d will belong to some other topic and hence d will have a higher probability for those topic. 
- So even if w gets added to t, it won’t be bringing many such documents to t.

In [54]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim
import warnings
import pickle

**The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. I use "gensim.corpora Dictionary" class for this.**

In [55]:
corpus_dict_filepath = os.path.join(scratch_directory, 'trigram_dict_all.dict')

In [56]:
# Create the LineSentence object
headline_trigrams = LineSentence(headlines_trigrams_filepath)

execute = False
if execute:
    # learn the dictionary by iterating over all of the headlines
    dictionary_trigrams = Dictionary(headline_trigrams)

    # Filter tokens that are very rare or too common in the dictionary (filter_extremes) and reassign integer ids (compactify)
    dictionary_trigrams.filter_extremes(no_below=20, no_above=0.4)
    dictionary_trigrams.compactify()

    # Save the corpus dictionary
    dictionary_trigrams.save(corpus_dict_filepath)

In [57]:
# load the finished dictionary from disk
dictionary_trigrams = Dictionary.load(corpus_dict_filepath)

- LDA assumes that documents are represented by the bag-of-words model. In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded.
- BOW vectors are sparsed (mostly 0s) and large.
- I generate a bag-of-words representation for each headline (implemented by the trigram_bow_generator function) Then I save the resulting bag-of-words representations of the headlines as a matrix (each row is a document, each column is a sparse vector for each token in the document).

In [58]:
bow_corpus_filepath = os.path.join(scratch_directory, 'bow_trigrams_corpus_all.mm') # file for the BOW matrix (represents documents)

In [59]:
# generator function to read headlines from a file and yield a bag-of-words representation
def bow_generator(filepath):
    for hl in LineSentence(filepath):
        yield dictionary_trigrams.doc2bow(hl) # for each document, get its bow representation using the corpora dictionary

In [60]:
execute = False
if execute:
    # Generate bag-of-words representations for all headlines and save them as a matrix
    MmCorpus.serialize(bow_corpus_filepath, bow_generator(headlines_trigrams_filepath))

In [61]:
# Load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(bow_corpus_filepath)

- With the bag-of-words corpus, we are start to learn our topic model from the headlines. We need to pass the bag-of-words matrix and Dictionary from our to the LdaMulticore function as inputs. 
- We also need to specify the number of topics the model should learn. 
- We specifiy K=4 topics (which coincide with the "true" number of topics that we have in our headline corpus).

In [62]:
set(label) # I have 4 "true" classes

{'Business', 'SciTech', 'Sports', 'World'}

In [63]:
lda_model_filepath = os.path.join(scratch_directory, 'lda_model_all')

In [64]:
 with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(
            trigram_bow_corpus,
            num_topics=4,
            id2word=dictionary_trigrams,
            workers=1
            )
lda.save(lda_model_filepath)

In [65]:
# load the trained LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

- Our topic model is now trained and ready to be used. 
- Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [66]:
# use a user-supplied topic number and print out a formatted list of the top tokens
def explore_topic(topic_number, topn=15):
    print(f'{"term":20} {"frequency"}' + '\n') # column headers
    
    for term, frequency in lda.show_topic(topic_number, topn):
        print(f'{term:20} {round(frequency, 3):.3f}')

In [67]:
set(label)

{'Business', 'SciTech', 'Sports', 'World'}

In [68]:
explore_topic(topic_number=0)   # Business

term                 frequency

ap                   0.015
team                 0.009
company              0.008
new                  0.006
end                  0.005
report               0.005
million              0.005
today                0.005
afp                  0.005
government           0.005
start                0.005
day                  0.005
on_tuesday           0.005
yesterday            0.004
microsoft            0.004


In [69]:
explore_topic(topic_number=1)    # World or Business?

term                 frequency

new                  0.015
reuters              0.006
lead                 0.006
yesterday            0.006
quot                 0.005
company              0.005
ap                   0.005
internet             0.005
time                 0.005
second               0.005
today                0.005
release              0.005
tuesday              0.005
country              0.004
base                 0.004


In [70]:
explore_topic(topic_number=2) # SciTech?

term                 frequency

company              0.014
win                  0.010
ap                   0.007
yesterday            0.007
year                 0.006
reuters              0.006
service              0.006
game                 0.005
announce             0.005
new                  0.005
million              0.004
report               0.004
on_wednesday         0.004
more_than            0.004
on_monday            0.004


In [71]:
explore_topic(topic_number=3) # sports

term                 frequency

reuters              0.007
new                  0.007
company              0.007
report               0.006
government           0.006
group                0.006
on_friday            0.005
big                  0.005
on_wednesday         0.004
quot                 0.004
country              0.004
record               0.004
ap                   0.004
leader               0.004
year                 0.004


**In case the news headlines are not labelled in this problem, then I can apply LDA to the corpus and examine the tokens that are grouped in each topic and figure out what the topic might be. Then I can labelled each of the documents/headlines with these topics.**

In [72]:
topic_names = {0: "Business", 1: "World", 2: "SciTech", 3:"Sports"}

- Manually review the top terms for each topic is a helpful exercise to figure out what the topic might be. 
- But to get a deeper understanding of the topics and how they relate to each other, we need to visualize the data — preferably in an interactive format.  - We use the pyLDAvis libary for that.  It is a library that takes topic models from gensim and prepare visualizations with the data.

In [73]:
LDAvis_data_filepath = os.path.join(scratch_directory, 'ldavis_prepared')

In [74]:
'''
The pyLDAvis.gensim.prepare function takes the gensim lda model, the bow representation of the documents (a matrix), and 
the corpus dictionary generated from gensim as inputs
'''
execute = False
if execute:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, dictionary_trigrams)

    with open(LDAvis_data_filepath, 'wb') as f:  # write binary (file mode)
        pickle.dump(LDAvis_prepared, f)

In [75]:
# load the prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

- Use the pyLDAvis display function to display the topic model visualization inline in the notebook

- The interactive visualization pyLDAvis produces is helpful for both:
    - (1) Better understanding and interpreting individual topics, and
    - (2) Better understanding the relationships between the topics.

- For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the $\lambda$ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

- For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

In [76]:
pyLDAvis.display(LDAvis_prepared)

## Text Description for LDA
- Besides "clustering" documents into topics, one of the main uses of LDA is to provide a compact and qunatitative description of text data.
- Once LDA model is trained, it can be used to represent documents as a mixture of topics (some with more weights/probability than others).
- This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% Topic A, 20% Topic B, 20% Topic C, and 10% Topic D.

In [77]:
def lda_description(headline_text, min_topic_freq=0.05):
    """
    1) accept the original text of a headline 
    2) parse it with spaCy
    3) apply text pre-proccessing steps
    4) create a bag-of-words representation of the document
    5) create an LDA representation of the document
    6) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the headline text with spaCy
    parsed_headline = nlp(headline_text)
    
    # lemmatize the text and remove punctuation and whitespace
    headline_unigrams = [
        lemm(token)
        for token in parsed_headline
        if (not punct_space(token) and get_alphanumeric(token))
        ]
    
    # apply the first-order and secord-order phrase models
    headline_bigrams = bigram_phrases_model[headline_unigrams]
    headline_trigrams = trigram_phrases_model[headline_unigrams]
    
    # remove any remaining stopwords
    headline_trigrams = [t for t in headline_trigrams if not t in nlp.Defaults.stop_words]
        
    # create a bag-of-words representation
    headline_bow = dictionary_trigrams.doc2bow(headline_trigrams)
    
    # create an LDA representation
    headline_lda = lda[headline_bow]
    
    # sort with the most highly related topics first
    headline_lda = sorted(headline_lda, key=lambda topic_number_freq: -topic_number_freq[-1])
    
    for topic_number, freq in headline_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print(f'{topic_names[topic_number]:25} {round(freq, 3):.3f}')

**Here I use "existing" headlines from the dataset.  However, if I have "new" headlines that are unlabelled, then I can use LDA (trained from existing labelled headlines) to estimate the label for the new unlabelled headlines.**

In [78]:
headline_num = 1452 

with open("news_article_data.json") as file:
    # convert the json record to a Python dict. "data" is a list of dicts
    data=[json.loads(line) for line in file]
    

sample_headline = data[headline_num]['content']
print(sample_headline)

The global extinction crisis is worse than thought, because thousands of  quot;affiliated quot; species also at risk do not figure in calculations.


In [79]:
lda_description(sample_headline)

Sports                    0.919


## Word Embeddings representations with word2vec
- The goal of word vector embedding models is to learn dense, numerical vector representations for each term in a corpus vocabulary.
- The vectors it learns about each term should encode some information about the meaning or concept the term represents, and the relationship between it and other terms in the vocabulary. 
- Word vector models are also fully unsupervised — they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.
- Perhaps the best-known word vector model is word2vec, originally proposed in 2013. The general idea of word2vec is, for a given focus word, to use the context of the word — i.e., the other words immediately before and after it — to provide hints about what the focus word might mean. To do this, word2vec uses a sliding window technique, where it considers snippets of text only a few tokens long at a time.

- At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. 
- The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. 
- Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. 
- One complete pass sliding the window across all of the corpus text is known as a training epoch. It's common to train a word2vec model for multiple passes/epochs over the corpus. 
- Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are close to each other in vector space.

- Word2vec has a number of user-defined hyperparameters, including:
    - The dimensionality of the vectors. I choose 300 for this example.
    - The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
    - The number of training epochs.

**If I want to train my own word2vec model with my corpus, gensim comes with a highly-optimized implementation word2vec algorithm with its Word2Vec class.**

In [80]:
from gensim.models import Word2Vec

sentences_trigrams = LineSentence(sentences_trigrams_filepath) # get LineSentence iterator of my headlines (with trigrams and stopwords removed)
word2vec_filepath = os.path.join(scratch_directory, 'word2vec_model_all')

In [81]:
execute = False

if execute:
    
    # initiate the model and perform the 20 epoch of training
    w2v = Word2Vec( 
        sentences_trigrams, # LineSentence object of the transformed headline corpus
        size=300, # size of the word vector
        window=5, # size of the window
        min_count=3, # Ignores all words with total frequency lower than this.
        sg=1, # Training algorithm: 1 for skip-gram; otherwise CBOW.
        workers=1, # Use these many worker threads to train the model
        iter=20 # number of epoch
        )
    
    # save model
    w2v.save(word2vec_filepath)

**The training is streamed, meaning sentences can be a generator, reading input data from disk on-the-fly, without loading the entire corpus into RAM. It also means you can continue training the model later:**

In [82]:
# load model
w2v = Word2Vec.load(word2vec_filepath)

print(f'The model was trained with {w2v.epochs} epochs.')

# train with new words if necessary
# w2v.train([['hello','world','I','am','here']], total_examples=1, epochs=1)

# The trained word vectors are stored in a KeyedVectors instance in w2v.wv:
# vector = w2v.wv['computer']  # numpy vector of a word


# Gensim instances of KeyedVectors (the common interface of sets of word-vectors) 
# contain a method init_sims(), which internally calculates unit-length 
# normalized vectors using a native vector operation for speed.
w2v.init_sims() # precompute the normalized vectors

The model was trained with 20 epochs.


The reason for separating the trained vectors into KeyedVectors is that if you don’t need the full model state any more (don’t need to continue training), the state can discarded, resulting in a much smaller and faster object that can be mmapped for lightning fast loading and sharing the vectors in RAM between processes:

In [83]:
# from gensim.models import KeyedVectors

# path = get_tmpfile("wordvectors.kv") # path the save model as a keyvector file
# word_vectors = KeyedVectors.load(word2vec_filepath, mmap='r')
# vector = word_vectors['computer']  # numpy vector of a word

In [84]:
print(f'{len(w2v.wv.vocab):,} terms in the w2v vocabulary, each with its 300x1 dense vector representations.')

561 terms in the w2v vocabulary, each with its 300x1 dense vector representations.


In [85]:
# Build a DataFrame that contains a set of word embeddings that I learnt
# I can get the index and count (frequency of word)
ordered_vocab=[(term, voc.index, voc.count) for term, voc in w2v.wv.vocab.items()]

In [86]:
ordered_vocab[:5]

[('union', 434, 61),
 ('worker', 294, 81),
 ('at', 15, 1070),
 ('say', 13, 1133),
 ('be', 6, 3754)]

In [87]:
# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda term_tuple: -term_tuple[2])

In [88]:
ordered_vocab[:5]

[('the', 0, 12464),
 ('a', 1, 6424),
 ('of', 2, 5085),
 ('to', 3, 4820),
 ('in', 4, 4548)]

In [89]:
# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

In [90]:
ordered_terms[:5]

('the', 'a', 'of', 'to', 'in')

In [91]:
term_indices[:5]

(0, 1, 2, 3, 4)

In [92]:
term_counts[:5]

(12464, 6424, 5085, 4820, 4548)

In [93]:
# create a DataFrame with the w2v vectors as data, and the terms as row labels
word_vectors_df = pd.DataFrame(
    w2v.wv.vectors_norm[term_indices, :],
    index=ordered_terms
    )

word_vectors_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
injury,-0.028678,0.003672,-0.046433,0.042980,-0.050665,-0.097703,0.064402,-0.070111,-0.047728,-0.100487,...,0.014500,-0.022889,0.005913,-0.019160,0.071120,0.065715,-0.033689,-0.061767,-0.013519,0.056492
question,0.015104,0.026804,0.020081,0.060929,0.007962,-0.014428,0.025457,0.004132,0.097565,-0.015763,...,-0.062230,-0.069003,-0.033870,-0.006200,0.068917,0.057053,-0.040870,-0.122160,-0.024437,-0.005812
airline,0.044040,-0.012662,0.028198,0.082535,-0.037324,-0.018668,0.063906,0.016214,-0.075572,-0.100135,...,-0.085299,-0.071514,-0.059542,-0.056584,-0.014410,0.062958,-0.046979,-0.038288,0.031768,0.111603
receive,-0.047220,-0.005763,0.046993,0.097229,0.050526,-0.048636,-0.066621,-0.022332,-0.029707,-0.148279,...,-0.025118,-0.040520,-0.031505,-0.143225,-0.014615,0.158997,0.005598,0.012431,0.008274,0.027896
tokyo,-0.054072,-0.102392,0.026237,0.039517,0.032118,-0.006891,-0.020864,0.052435,-0.102598,-0.110021,...,-0.029326,-0.031980,-0.043148,0.001361,-0.081997,0.072917,-0.051460,0.011985,0.001258,0.040532


**The w2v model has "embedded" the terms into a 300-dimensional vector space.**

In [94]:
# This function prints the 10 most similar words to the target word
def get_related_terms(token, top_n=10):

    for word, similarity in w2v.wv.most_similar(positive=[token], topn=top_n):

        print(f'{word:20} {round(similarity, 3)}')

In [95]:
# words similar in context to "software"
get_related_terms("software")

microsoft            0.601
server               0.595
computer             0.594
device               0.59
pc                   0.578
version_of           0.575
digital              0.565
product              0.551
intel                0.551
wireless             0.543


In [96]:
get_related_terms("score")

yard                 0.792
victory_over         0.766
minute               0.703
throw                0.684
defeat               0.682
win_over             0.627
sports_network       0.613
beat                 0.602
point                0.59
night                0.58


In [97]:
get_related_terms("tokyo")

near                 0.583
new_york_reuters     0.548
send                 0.545
japanese             0.52
trade                0.513
rebel                0.507
unit                 0.506
target               0.504
force                0.493
global               0.491


**I can add or subtract word embeddings:**
- Provide a set of words or phrases that you'd like to add or subtract.
- Look up the vectors that represent those terms in the word vector model.
- Add and subtract those vectors to produce a new, combined vector.
- Look up the most similar vector(s) to this new, combined vector via cosine similarity.
- Return the word(s) associated with the similar vector(s).

In [98]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = w2v.wv.most_similar(positive=add, negative=subtract, topn=topn) # get the resultant word embedding
    
    for term, similarity in answers:
        print(term)

In [99]:
word_algebra(add=['trade', 'new_york'])

tokyo


In [100]:
word_algebra(add=['computer', 'wireless'], subtract=['system'])

network


## Word Vector Visualization with t-SNE
- t-Distributed Stochastic Neighbor Embedding, or t-SNE for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

In [101]:
# scikit-learn has a TSNE class, we will use that implementation of t-sne
from sklearn.manifold import TSNE

**Our input for t-SNE will be the DataFrame of word vectors we created before. But beore that, we will:**
- Drop stopwords — it's probably not too interesting to visualize the, of, or, and so on


In [102]:
tsne_input = (word_vectors_df.drop(nlp.Defaults.stop_words, errors='ignore')) #drop stopwords from the word embeddings
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
company,-0.022861,-0.050449,0.001595,0.035906,0.096397,-0.034569,0.135193,0.068926,-0.020791,-0.061270,...,-0.013287,-0.100581,0.022391,-0.050550,0.023841,0.087962,0.017231,0.007809,0.009132,0.013007
new,0.039907,-0.026529,0.048099,-0.049626,0.114245,-0.084603,-0.062243,0.019767,0.012817,-0.024371,...,0.072619,-0.117317,-0.022663,-0.022308,-0.037683,0.077738,0.077305,-0.045504,-0.005399,-0.119434
ap,0.033210,0.013962,-0.063282,0.082089,-0.033795,-0.086432,-0.101568,-0.060251,0.016080,-0.090278,...,0.032503,0.004069,-0.017299,0.011773,-0.001047,0.078769,0.004370,-0.135862,0.022339,-0.077558
reuters,-0.035826,-0.120221,-0.046875,0.050515,-0.033914,0.000618,-0.061435,-0.067835,0.005880,-0.093429,...,0.015985,-0.075737,-0.038027,-0.056025,-0.032797,0.113574,-0.008005,-0.163231,0.056703,-0.078024
win,0.021388,0.031692,-0.012135,0.066727,0.079087,-0.052649,-0.011732,-0.106668,-0.040447,-0.039560,...,0.047772,-0.129981,0.052506,0.026554,-0.018875,0.082755,0.032691,-0.056480,-0.069349,0.018326


In [103]:
tsne_input.values # array of arrays

array([[-0.02286081, -0.0504493 ,  0.00159523, ...,  0.00780879,
         0.00913223,  0.01300692],
       [ 0.03990682, -0.02652893,  0.0480987 , ..., -0.04550416,
        -0.00539919, -0.11943393],
       [ 0.03321031,  0.01396197, -0.06328236, ..., -0.13586223,
         0.0223388 , -0.07755819],
       ...,
       [ 0.04403977, -0.01266175,  0.02819777, ..., -0.03828754,
         0.03176774,  0.11160297],
       [-0.04722004, -0.00576312,  0.04699276, ...,  0.01243058,
         0.00827442,  0.02789558],
       [-0.05407192, -0.10239225,  0.02623746, ...,  0.01198477,
         0.0012576 ,  0.04053189]], dtype=float32)

In [104]:
tsne_filepath = os.path.join(scratch_directory, 'tsne_model')
tsne_vectors_filepath = os.path.join(scratch_directory, 'tsne_vectors.npy')

In [105]:
execute = True
if execute:
    # init the tsne object
    tsne=TSNE() 
    tsne_vectors = tsne.fit_transform(tsne_input.values) # array of 300-D word embedding arrays as input
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f) # save the tsne model
        pd.np.save(tsne_vectors_filepath, tsne_vectors) # save the transformed tsne_vectors

In [106]:
# load the tsne_vectors
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f) # get the tsne model

tsne_vectors = pd.np.load(tsne_vectors_filepath) # get the tsne vectors    

# put tsne vectors into a DataFrame
tsne_vectors = pd.DataFrame(
    tsne_vectors,
    index=pd.Index(tsne_input.index), # row index are the tokens
    columns=['x_coord', 'y_coord']  # each token nly have 2 dimensions now 
    )

In [107]:
tsne_vectors.head()  # tsne vectors are 2-D represnetations of the 300D word-embedding vectors

,x_coord,y_coord
company,14.128726,-12.615202
new,13.363648,-6.801100
ap,1.764820,17.256807
reuters,-0.102177,-20.531179
win,9.933373,12.300522


In [108]:
tsne_vectors['word'] = tsne_vectors.index 
tsne_vectors.head()

,x_coord,y_coord,word
company,14.128726,-12.615202,company
new,13.363648,-6.801100,new
ap,1.764820,17.256807,ap
reuters,-0.102177,-20.531179,reuters
win,9.933373,12.300522,win


## Plotting Word Vectors with Bokeh

In [109]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

In [110]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(
    title='t-SNE Word Embeddings',
    plot_width=800,
    plot_height=800,
    tools=(
        'pan, wheel_zoom, box_zoom,'
        'box_select, reset'
        ),
    active_scroll='wheel_zoom'
    )

# add a hover tool to display words on roll-over
tsne_plot.add_tools(
    HoverTool(tooltips = '@word')
    )

# draw the words as circles on the plot
tsne_plot.circle(
    'x_coord',
    'y_coord',
    source=plot_data,
    color='blue',
    line_alpha=0.2,
    fill_alpha=0.1,
    size=10,
    hover_line_color='black'
    )

# configure visual elements of the plotc
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Text Categorization with spaCy's textcat

- https://www.kaggle.com/poonaml/text-classification-using-spacy
- https://datascience.stackexchange.com/questions/55896/how-to-train-a-spacy-model-for-text-classification

## Process data

**I need to wrangle the data to a form that is preferred by spAcy textcat**

In [411]:
with open("news_article_data.json") as file:
    first_record = file.readline()

print(first_record)

{"content": "Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.","annotation":{"notes":"","label":["Business"]},"extras":null,"metadata":{"first_done_at":1521027375000,"last_updated_at":1521027375000,"sec_taken":0,"last_updated_by":"nlYZXxNBQefF2u9VX52CdONFp0C3","status":"done","evaluation":"NONE"}}



In [412]:
set(label)

{'Business', 'SciTech', 'Sports', 'World'}

In [413]:
# a list of headlines that belong to one of the four classes
business_headlines = [] # a list of tuples of text and a dict of labels
scitech_headlines = []
sports_headlines = []
world_headlines = []

In [414]:
# lists that contain the text and the labels from all the headlines
all_content = []
all_type = []

In [415]:
with open('news_article_data.json') as file:
    for i, headline in enumerate(file):
        headline = json.loads(headline) # convert to dict
        
        # populate text and label from all headlines
        all_content.append(headline['content'])
        all_type.append(headline['annotation']['label'][0])
    
        # populate the list of (text, label) tuples in a format that is preferred by spaCy textcat
        if headline['annotation']['label'][0] =="Business":
            business_headlines.append((
            headline['content'], # text
            {'cats': {'business': 1.0, 'scitech': 0.0, 'sports': 0.0, 'world': 0.0}} # labels
            ))
            continue
            
        if headline['annotation']['label'][0] == "SciTech":
            scitech_headlines.append((
            headline['content'], # text
            {'cats': {'business': 0.0, 'scitech': 1.0, 'sports': 0.0, 'world': 0.0}} # labels
            ))
            continue
            
        if headline['annotation']['label'][0] == "Sports":
            sports_headlines.append((
            headline['content'], # text
            {'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 1.0, 'world': 0.0}} # labels
            ))
            continue
            
        if headline['annotation']['label'][0] == "World":
            world_headlines.append((
            headline['content'], # text
            {'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 0.0, 'world': 1.0}} # labels
            ))
            continue

**Take a look at a headline in spaCy's preferred format**

In [416]:
headline_text, headline_label = business_headlines[1]
print(headline_text)
print('')
print(headline_label)

Reuters - Apparel retailers are hoping their\back-to-school fashions will make the grade among\style-conscious teens and young adults this fall, but it could\be a tough sell, with students and parents keeping a tighter\hold on their wallets.

{'cats': {'business': 1.0, 'scitech': 0.0, 'sports': 0.0, 'world': 0.0}}


In [417]:
headline_text, headline_label = scitech_headlines[1]
print(headline_text)
print('')
print(headline_label)

AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.

{'cats': {'business': 0.0, 'scitech': 1.0, 'sports': 0.0, 'world': 0.0}}


In [418]:
headline_text, headline_label = sports_headlines[1]
print(headline_text)
print('')
print(headline_label)

FOXBOROUGH -- Looking at his ridiculously developed upper body, with huge biceps and hardly an ounce of fat, it's easy to see why Ty Law, arguably the best cornerback in football, chooses physical play over finesse. That's not to imply that he's lacking a finesse component, because he can shut down his side of the field much as Deion Sanders ...

{'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 1.0, 'world': 0.0}}


In [419]:
headline_text, headline_label = world_headlines[1]
print(headline_text)
print('')
print(headline_label)

AP - The man who claims Gov. James E. McGreevey sexually harassed him was pushing for a cash settlement of up to  #36;50 million before the governor decided to announce that he was gay and had an extramarital affair, sources told The Associated Press.

{'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 0.0, 'world': 1.0}}


In [420]:
len(business_headlines), len(scitech_headlines), len(sports_headlines), len(world_headlines)

(1900, 1900, 1900, 1900)

### In addition to using the original headline text as features, we also want to try using the "transformed" text as features (tokenized, lemmatized, stopwords and non-alphanumeric characters removed, bigrams and trigrams identified)

**Wrangle the transformed text in a format that is preferrable by textcat**

In [421]:
# get the processed text
transformed_content = []
with open(headlines_trigrams_filepath) as file:
    for headline in file:
        transformed_content.append(headline)

In [422]:
headline_df = pd.DataFrame(list(zip(all_content, transformed_content, all_type)), columns = ['content', 'transformed', 'type'])

In [423]:
headline_df.head()

,content,transformed,type
0,Unions representing workers at Turner Newall...,union represent worker turner newall disappoin...,Business
1,"SPACE.com - TORONTO, Canada -- A second\team o...",toronto canada rocketeer compete million_ansar...,SciTech
2,AP - A company founded by a chemistry research...,ap company found chemistry researcher universi...,SciTech
3,AP - It's barely dawn when Mike Fitzpatrick st...,ap barely dawn mike fitzpatrick start shift bl...,SciTech
4,AP - Southern California's smog-fighting agenc...,ap southern_california smog fighting agency em...,SciTech


In [424]:
headline_df["transformed"] = headline_df["transformed"].apply(lambda x: x.replace('\n',''))  # remove \n

In [425]:
# get DataFrames of transformed headline texts for each class
t_business_df = headline_df.loc[headline_df.type=="Business", ["transformed", "type"]]
t_scitech_df = headline_df.loc[headline_df.type=="SciTech",  ["transformed", "type"]]
t_sports_df = headline_df.loc[headline_df.type=="Sports",  ["transformed", "type"]]
t_world_df = headline_df.loc[headline_df.type=="World",  ["transformed", "type"]]

In [428]:
 # a list of tuples that contains the transformed text and its associated label in spaCy textcat's preferrable format
t_business_headlines = [] 
t_scitech_headlines = []
t_sports_headlines = []
t_world_headlines = []

for row in t_business_df.values.tolist():
    t_business_headlines.append((row[0], # text
                                 {'cats': {'business': 1.0, 'scitech': 0.0, 'sports': 0.0, 'world': 0.0}} )) # labels  
        
for row in t_scitech_df.values.tolist():
    t_scitech_headlines.append((row[0], # text
                                 {'cats': {'business': 0.0, 'scitech': 1.0, 'sports': 0.0, 'world': 0.0}} )) # labels  

        
for row in t_sports_df.values.tolist():
    t_sports_headlines.append((row[0], # text
                                 {'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 1.0, 'world': 0.0}} )) # labels
    
        
for row in t_world_df.values.tolist():
    t_world_headlines.append((row[0], # text
                                 {'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 0.0, 'world': 1.0}} )) # labels 

In [429]:
# take a look at a headline
text, cat = t_business_headlines[0]
print(text)
print('')
print(cat)

union represent worker turner newall disappoint talk_with stricken parent firm federal mogul

{'cats': {'business': 1.0, 'scitech': 0.0, 'sports': 0.0, 'world': 0.0}}


In [430]:
text, cat = t_scitech_headlines[0]
print(text)
print('')
print(cat)

toronto canada rocketeer compete million_ansari_x_prize contest fund suborbital space flight officially announce date manned rocket

{'cats': {'business': 0.0, 'scitech': 1.0, 'sports': 0.0, 'world': 0.0}}


In [431]:
text, cat = t_sports_headlines[0]
print(text)
print('')
print(cat)

michael_phelps win gold_medal 400 individual medley set world_record time 4 minute second

{'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 1.0, 'world': 0.0}}


In [432]:
text, cat = t_world_headlines[0]
print(text)
print('')
print(cat)

canadian_press vancouver cp sister man_who die violent confrontation police demand city chief constable resign defend officer involve

{'cats': {'business': 0.0, 'scitech': 0.0, 'sports': 0.0, 'world': 1.0}}


### We'll split the data 50/50 into train and test sets, then randomly shuffle each set.

In [397]:
import random

In [398]:
train_data = business_headlines[:int(len(business_headlines) / 2)] + scitech_headlines[:int(len(scitech_headlines) / 2)] + sports_headlines[:int(len(sports_headlines) / 2)] + world_headlines[:int(len(world_headlines) / 2)]
test_data = business_headlines[int(len(business_headlines) / 2):] + scitech_headlines[int(len(scitech_headlines) / 2):] + sports_headlines[int(len(sports_headlines) / 2):] + world_headlines[int(len(world_headlines) / 2):] 

In [399]:
random.shuffle(train_data)
random.shuffle(test_data)

In [400]:
len(train_data), len(test_data)

(3800, 3800)

**Repeat the same thing for the transformed data**

In [436]:
t_train_data = t_business_headlines[:int(len(t_business_headlines) / 2)] + t_scitech_headlines[:int(len(t_scitech_headlines) / 2)] + t_sports_headlines[:int(len(t_sports_headlines) / 2)] + t_world_headlines[:int(len(t_world_headlines) / 2)]
t_test_data = t_business_headlines[int(len(t_business_headlines) / 2):] + t_scitech_headlines[int(len(t_scitech_headlines) / 2):] + t_sports_headlines[int(len(t_sports_headlines) / 2):] + t_world_headlines[int(len(t_world_headlines) / 2):] 

In [437]:
random.shuffle(t_train_data)
random.shuffle(t_test_data)

In [438]:
len(t_train_data), len(t_test_data)

(3800, 3800)

**Next, we'll create a new textcat model and add it to our existing nlp spaCy pipeline.**

In [401]:
original_pipe_names = nlp.pipe_names
original_pipe_names

['tagger', 'parser', 'ner']

In [402]:
# create the textcat model, default model that it uses is CNN
textcat = nlp.create_pipe(
    'textcat',
    config={'exclusive_classes': True} # classes are mutually exclusive (binary classification)
    )

# add labels to the model
textcat.add_label('business')
textcat.add_label('scitech')
textcat.add_label('sports')
textcat.add_label('world')

1

In [403]:
# add textcat model to the pipe
nlp.add_pipe(textcat)
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [404]:
# This function helps us evaluate the performance of our text classification model
def evaluate(tokenizer, textcat, texts, cats):
    
    docs = (tokenizer(text) for text in texts)
    
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    
    for i, doc in enumerate(textcat.pipe(docs)):

        gold = cats[i]['cats'] # get the labels for each headline in the test dataset (ex: {Business:1, SciTech:0, Sports:0, World:0})
        
        # label and score and the predicted z-score for each label
        for label, score in doc.cats.items(): 
            #print((label, score))
            
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    
    # Compute the estimated test precision, recall and f1-score (from the validation-set approach)
    precision = tp / (tp + fp)  # I use micro-averaging method here
    recall = tp / (tp + fn)
    
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

**We start the training loop for our textcat model, using train_data for training the model and test_data for evaluating its performance.**

In [405]:
from spacy.util import minibatch

In [406]:
with nlp.disable_pipes(*original_pipe_names): # disable ner, tagger, and parser, leaving only textcat enabled in nlp 
    
    optimizer = nlp.begin_training() # initialize weights in CNN model (the default model of textcat)
    
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    
    for i in range(10):
        losses = {}

        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)

        batches = minibatch(train_data, size=8)

        for batch in batches:
            texts, cats = zip(*batch)  # train text and labels
            nlp.update(texts, cats, sgd=optimizer, drop=0.2, losses=losses) # update model with new train data

        with textcat.model.use_params(optimizer.averages):

            # evaluate model on the test data 
            test_texts, test_cats = zip(*test_data)
            scores = evaluate(nlp.tokenizer, textcat, test_texts, test_cats)

        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table of train loss, and test precision, recall and f-score of each epoch
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
                )
            )

Training the model...
LOSS 	  P  	  R  	  F  
4.799	0.735	0.335	0.460
3.058	0.809	0.707	0.755
1.779	0.828	0.777	0.801
1.156	0.821	0.786	0.803
0.709	0.818	0.793	0.805
0.487	0.811	0.788	0.799
0.381	0.810	0.791	0.800
0.227	0.808	0.794	0.801
0.189	0.809	0.798	0.803
0.196	0.816	0.802	0.809


**Train and assess model using transformed data**

In [439]:
with nlp.disable_pipes(*original_pipe_names):
    
    optimizer = nlp.begin_training()
    
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    
    for i in range(10):
        losses = {}

        # batch up the examples using spaCy's minibatch
        random.shuffle(t_train_data)

        batches = minibatch(t_train_data, size=8)

        for batch in batches:
            texts, cats = zip(*batch)
            nlp.update(texts, cats, sgd=optimizer, drop=0.2, losses=losses)

        with textcat.model.use_params(optimizer.averages):

            # evaluate on the dev data split off in load_data()
            test_texts, test_cats = zip(*t_test_data)
            scores = evaluate(nlp.tokenizer, textcat, test_texts, test_cats)

        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
                )
            )

Training the model...
LOSS 	  P  	  R  	  F  
2.005	0.789	0.741	0.764
0.876	0.791	0.757	0.774
0.493	0.785	0.763	0.773
0.292	0.778	0.763	0.771
0.242	0.775	0.761	0.768
0.193	0.773	0.761	0.767
0.141	0.773	0.762	0.767
0.096	0.772	0.764	0.768
0.123	0.770	0.761	0.765
0.106	0.765	0.756	0.761


## Thoguhts:
- If the lemmatized, cleaned, and transformed text is used to trained the CNN model, its performance is actually worse than if I used the original text.
- The train loss decreased at each epoch while the test metrics do not change much, this indicates overfitting.  I might only need 10 epoches since the test metrics do not change much.